In [242]:
import numpy as np
import pandas as pd
import copy

In [243]:
def validity_check(image, conv_filter, padding, stride):
    
    is_valid = True
    image_shape = (np.shape(image))
    
    filter_shape = (np.shape(conv_filter))
    
    
    if len(image_shape) != len(filter_shape) - 1:
        print('Dimensions of both image and filter should be equal')
        is_valid = False
    
    if len(image_shape) >= 3 or len(filter_shape) > 3:
        if image_shape[-1] != filter_shape[-1]:
            print('Number of channels in both image and filter should be equal')
            is_valid = False
        
    if filter_shape[1] != filter_shape[2]:
        print("Filter should be of a square matrix")
        is_valid = False
        
        

    if filter_shape[1] % 2 == 0:
        print('dimensions of filter should be of odd dimensions not even')
        is_valid = False
    

    '''
    convolution_result is the array we obtain after running the filter all over the image
    '''    
    if is_valid:
        '''one or more filters but only one channel (also for image)'''
        if len(image_shape) == 2 and len(filter_shape) == 3:
            
            convolution_result = np.zeros((
                            np.int16(((image_shape[0] + 2 * padding - filter_shape[1]) / stride) + 1),
                            np.int16(((image_shape[1] + 2 * padding - filter_shape[2]) / stride) + 1)))
            
        '''one or more filters with more than one channel (also for image)'''
        if len(image_shape) == 3 and len(filter_shape) == 4:
            convolution_result = np.zeros((
                            np.int16(((image_shape[0] + 2 * padding - filter_shape[1]) / stride) + 1),
                            np.int16(((image_shape[1] + 2 * padding - filter_shape[2]) / stride) + 1),
                            image_shape[3]))
    else:
        convolution_result = -1
    
    return [is_valid, convolution_result]


def Relu(x):
    indices = np.where(x < 0)
    x[indices] = 0
    return x

In [244]:
'''
The dimensions represent height, width and no_of_channels in the input image and filters
'''


class Conv_pool:
    
    def __init__(self, image, conv_filter, padding, stride, pooling_filter_size, pooling_filter_stride):
        self.curr_image         = image
        self.conv_filter        = conv_filter
        self.conv_filter_stride = stride
        self.pool_filter_size   = pooling_filter_size
        self.pool_filter_stride = pooling_filter_stride
        self.padding            = padding
        self.conv_result        = np.array([])
        self.relu_result        = np.array([])
        self.pooling_result     = np.array([])
            

    ######################################################
    
    def start_convolving_the_image(self, convolution_result):
        image = self.curr_image
        conv_filter = self.conv_filter
        stride = self.conv_filter_stride
        convoluted_image_list = []

        image_shape = np.shape(image)
        filter_shape = np.shape(conv_filter)

        no_of_filters = filter_shape[0]

        '''this loop runs for all the different filters in the conv_filter'''
        for filter_num in range(0, no_of_filters):
            convoluted_image = np.array([])
            filter_ = conv_filter[filter_num, :]
    
            '''if image has more than one channel'''
            if len(image_shape) > 2 and image_shape[-1] == filter_shape[-1]:

                convoluted_image = self.convolve_the_image_by_filter(image[:, :, 0], filter_[:, :, 0], 
                                                                   convolution_result)
                for channel in range(1, filter_shape[-1]):
                    convoluted_image = convoluted_image + self.convolve_the_image_by_filter(image[:, :, channel], filter_[:, :, channel],
                                                                convolution_result)
            '''if image has only one channel'''
            if len(image_shape) == 2:
                convoluted_image = self.convolve_the_image_by_filter(image, filter_, convolution_result)
            '''here, you used copy.copy() so as to refrain the convoluted_image_list getting over-written
            by the new entries into the list'''
            convoluted_image_list.append(copy.copy(convoluted_image))

        self.conv_result = np.array(convoluted_image_list)

        

        
    ######################################################
    
    def convolve_the_image_by_filter(self, image, conv_filter, convolution_result):
        stride = self.conv_filter_stride
        image_shape = np.shape(image)    
        filter_size = np.shape(conv_filter)[0]
        row_count = 0
        column_count = 0
        for row in range(0, image_shape[0], stride):
            if row + filter_size <= image_shape[0]:
                column_count = 0
                for column in range(0, image_shape[1], stride):

                    if column + filter_size <= image_shape[1]:

                        '''present active region is the region in the image which will be multiplied
                        by the filter'''
                        present_active_region = image[row : (row + filter_size), column : (column + filter_size)]
                        '''element-wise multiplication between image region and filter'''
                        image_area_product_filter = present_active_region * conv_filter
                        sum_of_all_elements = np.sum(image_area_product_filter)

                        convolution_result[row_count, column_count] = sum_of_all_elements
                        column_count = column_count + 1
            row_count = row_count + 1

        return convolution_result

    
    
    ######################################################
    
    def relu_on_convolution_result(self):
        convolution_result_ = self.conv_result
        relu_result_list = []
        no_of_times_image_convoluted = np.shape(convolution_result_)[0]

        for result_num in range(0, no_of_times_image_convoluted):
            current_convoluted_image = convolution_result_[result_num, :]
            relu_result_list.append(Relu(current_convoluted_image))

        self.relu_result = np.array(relu_result_list)

        
    ######################################################
    
    '''max pooling'''    
    def pooling_on_relu_result(self):
        relu_result = self.relu_result
        pool_size = self.pool_filter_size
        stride_length = self.pool_filter_stride
        pooling_result_list = []
        no_of_relu_results = np.shape(relu_result)[0]
        row_count = 0
        column_count = 0
        relu_result_shape = np.shape(relu_result)[1]




        for relu_result_num in range(0, no_of_relu_results):
            pooling_result = np.zeros((np.uint16(((relu_result_shape - pool_size) / stride_length) + 1),
                                   np.uint16(((relu_result_shape - pool_size) / stride_length) + 1)))
            curr_relu_result = relu_result[relu_result_num, :]
            row_count = 0
            for row in range(0, relu_result_shape, stride_length):
                if row + pool_size <= relu_result_shape:
                    column_count = 0
                    for column in range(0, relu_result_shape, stride_length):
                        if column + pool_size <= relu_result_shape:
                            active_region = curr_relu_result[row : (row + pool_size), column : (column + pool_size)]
                            pooling_result[row_count, column_count] = np.max(active_region)
                            column_count = column_count + 1

                row_count = row_count + 1
            pooling_result_list.append(pooling_result)


        self.pooling_result = np.array(pooling_result_list)
        
    
    

    ######################################################
    
    def CNN(self):
    
        validity_convolution_result = validity_check(self.curr_image, self.conv_filter, self.padding, self.conv_filter_stride)

        if validity_convolution_result[0]:
            convolution_result = validity_convolution_result[1]
            self.start_convolving_the_image(convolution_result)
            self.relu_on_convolution_result()
            self.pooling_on_relu_result()

                    

In [328]:
def Relu(x):
    return x * (x > 0)
    
def softmax(x):
    x = x.reshape(1, -1)
    
    max_item = np.max(x)
    print('max is ' + str(max_item))
    exp_x = np.exp(x - max_item)
    res = np.true_divide(exp_x, np.sum(exp_x))
    return res



def cross_entropy(output, labels):
    return -np.sum(labels * np.log(output))



class fully_connected_layers_in_CNN:
    def __init__(self, conv_pool_object, labels, no_of_neurons):
        self.conv_pool_result = conv_pool_object
        self.pooling_result = conv_pool_object.pooling_result
        self.labels = labels
        self.pooling_result_shape = np.shape(self.pooling_result)
        self.neurons = no_of_neurons
    
    ##########################################################
    
    def feed_forward_in_fc_layers(self):
        pooling_result = self.pooling_result
        no_of_neurons = self.neurons
        no_of_pool_results, no_of_rows, no_of_columns = self.pooling_result_shape
        self.fc_layer_1 = pooling_result.reshape(((no_of_pool_results * no_of_rows * no_of_columns), 1))
        
        no_of_rows_in_fc_layer_1 = self.fc_layer_1.shape[0]
        self.weights_1 = np.random.random((no_of_neurons[0], no_of_rows_in_fc_layer_1))
        self.bias_1 = np.zeros(shape = (no_of_neurons[0], 1))
        
        self.z_1 = np.add(np.dot(self.weights_1, self.fc_layer_1), self.bias_1)
        print(np.shape(self.z_1))
        self.fc_layer_2 = Relu(self.z_1)
        
        no_of_rows_in_fc_layer_2 = self.fc_layer_2.shape[0]
        self.weights_2 = np.random.random((no_of_neurons[1], no_of_rows_in_fc_layer_2))
        self.bias_2 = np.zeros(shape = (no_of_neurons[1], 1))
        
        self.z_2 = np.add(np.dot(self.weights_2, self.fc_layer_2), self.bias_2)
        
        self.predicted_probs = softmax(self.z_2)
        print('pr ' + str(self.predicted_probs))
        self.loss = cross_entropy(self.predicted_probs, self.labels)
        self.diff_between_pred_and_actual_labels = self.predicted_probs - self.labels

    
    

In [326]:

class back_propagation_in_CNN:
    
    
    ##########################################################
    
    def __init__(self, fc_layer_object):
        self.fc_layer_object = fc_layer_object
        self.conv_pool_result = fc_layer_object.conv_pool_result
    
    
    
    
    ##########################################################
    
    def get_the_index(self, conv_sub_array):
        position_of_max_element = np.nanargmax(conv_sub_array)
        index_of_max_element = np.unravel_index(position_of_max_element, conv_sub_array.shape)
        return index_of_max_element
    
    
    ##########################################################
    
    def backpropagation_wrt_pooling_result(self, derivative_pool_result):
        
        
        conv_result = self.conv_pool_result.conv_result
        pool_filter_size = self.conv_pool_result.pool_filter_size
        pool_filter_stride = self.conv_pool_result.pool_filter_stride
        
        no_of_convolved_images, no_of_rows, no_of_columns = np.shape(conv_result)
        _, pool_row_length, _ = np.shape(derivative_pool_result)
        output = np.zeros(shape = np.shape(conv_result))
    
        row_count = 0
        column_count = 0
        for image_num in range(0, no_of_convolved_images):
            current_image = conv_result[image_num, :]
            row_count = 0
            for row in range(0, no_of_rows, pool_filter_stride):
                if row + pool_filter_size <= no_of_rows:
                    column_count = 0
                    for column in range(0, no_of_columns, pool_filter_stride):
                        if column + pool_filter_size <= no_of_columns:
                            image_sub_array = current_image[row : row + pool_filter_size, column : column + pool_filter_size]
                            '''index of the max element in the original conv_image (in the area covered by pool filter)'''
                            (i, j) = self.get_the_index(image_sub_array)
                            if row_count <= pool_row_length:
                                output[image_num, row + i, column + j] = derivative_pool_result[image_num, 
                                                                                            row_count, column_count]
                            column_count = column_count + 1
                
                row_count = row_count + 1
            
        return output
                              
    
    
    ##########################################################
    
    def backpropagation_wrt_convolution_layer(self, der_conv_prev):
                
        conv_curr_image = self.conv_pool_result.curr_image
        conv_image_curr_list = [conv_curr_image]
        conv_image_curr = np.array(conv_image_curr_list)
        conv_filter = self.conv_pool_result.conv_filter
        filter_stride = self.conv_pool_result.conv_filter_stride
        no_of_filters, flt_size, no_of_columns_f = np.shape(conv_filter)
        
        len_of_shape_of_curr_image = len(np.shape(conv_image_curr))
        
        _,no_of_rows, no_of_columns = np.shape(conv_image_curr)

        
        der_wrt_filter = np.zeros(np.shape(conv_filter))
        der_wrt_bias = np.zeros((no_of_filters, 1))
        dconv_result = np.zeros(np.shape(conv_image_curr))
        
        r_count = 0
        c_count = 0
        
        for filter_num in range(0, no_of_filters, filter_stride):
            r_count = 0
            for r in range(0, no_of_rows):
                if r + flt_size <= no_of_rows:
                    c_count = 0
                    for c in range(0, no_of_columns, filter_stride):
                        if c + flt_size <= no_of_columns:
                            '''update the filter'''
                            
                            der_wrt_filter += der_conv_prev[filter_num, r_count, c_count] * conv_image_curr[:, 
                                                                                                         r : r + flt_size,
                                                                                                         c : c + flt_size]
                            
                            dconv_result[:, r : r + flt_size, c : c + flt_size] += der_conv_prev[filter_num, r_count, c_count] * conv_filter[filter_num, :]
                            
                            c_count = c_count + 1
                r_count = r_count + 1
        
            der_wrt_bias[filter_num] = np.sum(der_conv_prev[filter_num])
        
        
        return dconv_result, der_wrt_filter, der_wrt_bias
                                
    
    
    
    ##########################################################
    
    def start_back_propagation(self):
        fc_layer_object = self.fc_layer_object
        fc_layer_1 = fc_layer_object.fc_layer_1
        self.derivative_wrt_weights_2  = np.dot(fc_layer_object.diff_between_pred_and_actual_labels.T, 
                                                       fc_layer_object.z_1.T)
        
        temporary_result = np.dot(fc_layer_object.weights_2.T,  
                                                    fc_layer_object.diff_between_pred_and_actual_labels.T)
        
        print(fc_layer_object.fc_layer_1)
        self.derivative_wrt_weights_1 = np.dot(temporary_result,
                                                  fc_layer_object.fc_layer_1.T)
        
        self.derivative_wrt_bias_2 = np.sum(fc_layer_object.diff_between_pred_and_actual_labels, 
                                                axis = 1).T
        self.derivative_wrt_bias_1 = np.sum(temporary_result, axis = 1).reshape(fc_layer_object.bias_1.shape)
        
        
        der_fc_layer_1 = np.dot(fc_layer_object.weights_1.T, temporary_result)
        reshaped_dfc_layer_1 = der_fc_layer_1.reshape(fc_layer_object.pooling_result_shape)
        '''back propagating through max-pooling layer (updating only those neurons with highest value)'''
        der_pool_layer = self.backpropagation_wrt_pooling_result(reshaped_dfc_layer_1)
        
        '''back propagating through Relu layer'''
        der_pool_layer[fc_layer_object.conv_pool_result.conv_result <= 0] = 0
        
        der_image, der_filter, der_conv_bias = self.backpropagation_wrt_convolution_layer(der_pool_layer)
        
        self.return_result = [der_filter, der_conv_bias ]
        
    
